**Filtragem de dados da planilha**

IMPORTAR PLANILHA PRO PANDAS

In [ ]:
import pandas as pd
import re


def extract_and_clean_year(row):
    # Tentamos encontrar o padrão (XX/XX) ou XX/XX
    match = re.search(r'(\((\d{2}/\d{2})\)|(\d{2}/\d{2}))', row['Modelo'])
    if match:
        # Verifica qual dos padrões foi encontrado e pega o valor correto
        year = match.group(2) if match.group(2) else match.group(3)

        # Limpa a string da coluna 'Modelo' removendo o padrão de ano
        row['Modelo'] = row['Modelo'].replace(match.group(0), '').strip()
        row['Ano'] = year
    else:
        row['Ano'] = '23/23'
    return row


# Carregar o arquivo Excel
excel_file = pd.ExcelFile('D:/tabelaFiat1.xlsx')
all_sheet_names = excel_file.sheet_names

# Processar cada planilha e extrair os dados
all_sheets_data = []
for current_sheet_name in all_sheet_names[1:]:
    current_sheet_data = pd.read_excel(excel_file, sheet_name=current_sheet_name, header=None)
    sheet_rows_data = []

    current_title = None  # Aqui guardaremos o título/modelo atual
    for _, row in current_sheet_data.iterrows():
        # Verifica se encontramos um novo título
        if pd.notna(row[0]) and pd.isna(row[1]) and pd.isna(row[2]) and pd.isna(row[3]):
            current_title = row[0]
        # Caso contrário, verifica se as colunas 1, 2 e 3 têm dados (não são NA)
        elif pd.notna(row[1]) and pd.notna(row[2]) and pd.notna(row[3]):
            row_data = pd.DataFrame([[row[1], row[2], row[3], current_title]], columns=['Peça', 'Código', 'Preço', 'Modelo'])
            sheet_rows_data.append(row_data)

    all_sheets_data.append(pd.concat(sheet_rows_data))

# Concatenar todos os dataframes
df_excel = pd.concat(all_sheets_data)
0
# Limpar e formatar o dataframe
df_excel['Preço'] = pd.to_numeric(df_excel['Preço'], errors='coerce')
df_excel.dropna(subset=['Preço'], inplace=True)
df_excel['Preço'] = df_excel['Preço'].round(2)
df_excel['Modelo'] = df_excel['Modelo'].str.replace('PALIO ', '', regex=False)
df_excel['Código'] = df_excel['Código'].astype(str)



# Aplicamos a função ao dataframe
df_excel = df_excel.apply(extract_and_clean_year, axis=1)

# Dividindo a coluna 'Ano' e criando 'ANO_INI' e 'ANO_FIM'.
df_excel[['ANO_INI', 'ANO_FIM']] = df_excel['Ano'].str.split('/', expand=True)

# Adicionando "20" na frente dos valores de 'ANO_INI' e 'ANO_FIM'
df_excel['ANO_INI'] = '20' + df_excel['ANO_INI']
df_excel['ANO_FIM'] = '20' + df_excel['ANO_FIM']


# Removendo a coluna 'Ano' original.
df_excel.drop('Ano', axis=1, inplace=True)

df_excel



**Fazer conexao com o banco**

In [ ]:
import pyodbc
import pandas as pd

# Substitua os seguintes valores pelas suas credenciais de banco de dados
server = 'RAFAELPC' 
database = 'dbfacamp' 
driver = '{ODBC Driver 17 for SQL Server}'

# String de conexão
conn_string = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Conecte ao banco de dados
conn = pyodbc.connect(conn_string)

# Crie um cursor a partir da conexão
cursor = conn.cursor()

Pegar dados dos produtos dos kits

In [ ]:
# Consulta SQL que você deseja executar
query = '''
SELECT
    t1.ID_KIT, t1.ID_PRODUTO, t1.VAL_UNITARIO,
    t2.PADRAO_DESCRICAO, t2.ANO_INI, t2.ANO_FIM,
    t3.DESCRICAO, t3.REFERENCIA
FROM KITS_PROD t1
JOIN KITS t2
    ON t1.ID_KIT = t2.ID_KIT
JOIN PRODUTOS t3
    ON t1.ID_PRODUTO = t3.ID_PRODUTO
'''

# Execute a consulta e coloque os resultados em um DataFrame
df_sql = pd.read_sql(query, conn)

# Feche o cursor e a conexão


# Exiba o DataFrame
df_sql


**Organizar os id's dos produtos com as referencias**

In [ ]:
df_sql_filtered = df_sql[['ID_PRODUTO', 'REFERENCIA']].drop_duplicates()
df_sql_filtered['REFERENCIA'] = df_sql_filtered['REFERENCIA'].astype(str)
df_sql_filtered = df_sql_filtered.reset_index(drop= True)
df_sql_filtered

**Colocar o idproduto na tabela df_excel**

In [ ]:
# Renomeando a coluna REFERENCIA para Código para fazer o merge
df_sql_filtered_renamed = df_sql_filtered.rename(columns={'REFERENCIA': 'Código'})

# Merge dos DataFrames com base na coluna 'Código'
df_combined = pd.merge(df_excel, df_sql_filtered_renamed, on='Código', how='left')


**COMBINAR AS TABELAS**

In [ ]:
df_combined['ID_PRODUTO'] = df_combined['ID_PRODUTO'].astype(str).str.replace('.0', '', regex=False)
df_combined = df_combined.dropna(subset=['ID_PRODUTO'])


df_combined

**Criar tabela temporaria no banco**

In [ ]:
# Primeiro, crie uma tabela temporária para armazenar os dados do DataFrame
cursor.execute('''
    CREATE TABLE TempUpdateTable (
        ID_PRODUTO INT,
        REFERENCIA NVARCHAR(50),
        MODELO NVARCHAR(50),
        ANO_INI NVARCHAR(10),
        ANO_FIM NVARCHAR(10),
        NOVO_VAL_UNITARIO DECIMAL(10, 2)
    )
''')
conn.commit()

**INSERIR OS DADOS TEMPORARIOS**

In [ ]:

# Agora, insira os dados do DataFrame na tabela temporária
for index, row in df_combined.iterrows():
    try:
        cursor.execute('''
            INSERT INTO TempUpdateTable (ID_PRODUTO, REFERENCIA, MODELO, ANO_INI, ANO_FIM, NOVO_VAL_UNITARIO)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', row['ID_PRODUTO'], row['Código'], row['Modelo'], row['ANO_INI'],row['ANO_FIM'], row['Preço'])
        conn.commit()
    except pyodbc.DataError as e:
        print(f"Erro ao inserir dados: {row['ID_PRODUTO'], row['Código'], row['Modelo'], row['ANO_INI'],row['ANO_FIM'], row['Preço'], e}")
        # Se desejar, você pode fazer algo com a linha que causou o erro aqui
        continue # Isso irá ignorar o erro atual e continuar com a próxima iteração



**Atualizar a tabela principal e deletar a temporaria**

In [ ]:
# Depois, atualize a tabela original usando um JOIN com a tabela temporária
cursor.execute('''
               
UPDATE t1
SET t1.VAL_UNITARIO = temp.NOVO_VAL_UNITARIO
FROM KITS_PROD t1
JOIN KITS t2 ON t1.ID_KIT = t2.ID_KIT
JOIN TempUpdateTable temp ON t1.ID_PRODUTO = temp.ID_PRODUTO
WHERE LOWER(t2.PADRAO_DESCRICAO) LIKE '%' + LOWER(temp.MODELO) + '%'
  AND t2.ANO_INI >= temp.ANO_INI
  AND t2.ANO_FIM <= temp.ANO_FIM;

''')
conn.commit()

# Não esqueça de remover a tabela temporária após a atualização
cursor.execute('DROP TABLE TempUpdateTable')
conn.commit() 
# Fechar a conexão
